# Installations:

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports:

In [ ]:
import pandas as pd

import torch_geometric
import numpy as np
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as Data
import torch_geometric.transforms as transforms
import networkx as nx
from torch_geometric.utils.convert import to_networkx
#from torch_geometric.data import HeteroData

# Loading the data:

In [ ]:
path = '/content/drive/MyDrive/Test.csv'

df = pd.read_csv(path, index_col=None, header=0, encoding="utf-8")
temp = df.head(1).values.tolist()[0]
labels = []
for i in range(1, 54, 3):
  labels.append(temp[i])

df = pd.read_csv(path, index_col=None, header=1, encoding="utf-8")

<ipython-input-3-06a85721056a>:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, index_col=None, header=0, encoding="utf-8")
<ipython-input-3-06a85721056a>:9: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, index_col=None, header=1, encoding="utf-8")


In [ ]:
Xs = []
Ys = []
CLs = []

for l in labels:
  Xs.append(df[l].values.tolist()[1:])
  Ys.append(df[l+'.1'].values.tolist()[1:])
  CLs.append(df[l+'.2'].values.tolist()[1:])

In [ ]:
node_mapping = dict()

for idx, lab in enumerate(labels):
  node_mapping['lab'] = idx

# Modeling the graph:

In [ ]:
def set_edges(epoch_len, land_mark_num, encoders=None):
    src = []
    dst = []

    for n in range(epoch_len):
      for i in range(land_mark_num):
        src.append(n*land_mark_num + i)
        dst.append(n*land_mark_num + i + land_mark_num)

      for i in range(land_mark_num):
        for j in range(land_mark_num):
          if i != j:
            src.append(i + n*land_mark_num)
            dst.append(j + n*land_mark_num)

    edge_index = torch.tensor([src, dst])

    edge_attr = None
    # if encoders is not None:
    #     edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
    #     edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [ ]:
def set_nodes(Xs, Ys, CLs, off_idx, land_mark_num, epoch_len, encoders=None):
  x = torch.tensor([])

  for n in range(epoch_len):
    for i in range(land_mark_num):
      step = torch.tensor([[float(Xs[i][n + off_idx])], 
                           [float(Ys[i][n + off_idx])], 
                           [float(CLs[i][n + off_idx])]], 
                          dtype=torch.float)
      x = torch.cat((x, step.T), dim=0)
  return x

In [ ]:
num_landmarks = 18
num_layers = 120
edge_index = []

# Connect each landmark to all other landmarks in the same layer
for layer in range(num_layers):
    start = layer * num_landmarks
    layer_edges = [(i, j) for i in range(start, start + num_landmarks) for j in range(start, start + num_landmarks) if i != j]
    layer_edges = [[edge[0], edge[1]] for edge in layer_edges]
    edge_index.extend(layer_edges)

# Connect each landmark to the same landmark in the next layer
for layer in range(num_layers - 1):
    start1 = layer * num_landmarks
    start2 = (layer + 1) * num_landmarks
    layer_edges = [[i, j] for i in range(start1, start1 + num_landmarks) for j in range(start2, start2 + num_landmarks) if i % num_landmarks == j % num_landmarks]
    edge_index.extend(layer_edges)
    
edge_index = torch.tensor(edge_index).t().contiguous()


In [ ]:
from torch_geometric.data import Data

epoch_len = 120
land_mark_num = 18
len = df.shape[0]-1
overlap = 0.5
graph_list = []
order = []
count = 0
step = int(epoch_len*(1-overlap))

#edge_index, edge_attr = set_edges(epoch_len, land_mark_num)

for i in range(1, len+1-2*step, step):
  x = set_nodes(Xs, Ys, CLs, i, land_mark_num, epoch_len)
  data = Data(x=x, edge_index=edge_index)
  graph_list.append(data)
  count = count + 1
  order.append((count, i))

* **Create and save the dataset:**

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

loader = DataLoader(graph_list, batch_size=16)
torch.save(loader, '/content/drive/MyDrive/Test_loader.pth')

pd.DataFrame(order).to_csv('/content/drive/MyDrive/order.csv')

In [ ]:
from torch_geometric.data import Data, DataLoader, InMemoryDataset

dataset = InMemoryDataset(root='/content/', transform=None)
dataset.data = graph_list
dataset.index_select = None

# Save the dataset to disk
torch.save(dataset, './dataset.pt')

# Load the dataset from disk
dataset2 = torch.load('./dataset.pt')

# Print the first data object in the dataset
print(dataset2[0])

[Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 38862]), Data(x=[2160, 3], edge_index=[2, 

In [ ]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url

root = '/content/drive/MyDrive/Test_loader.pth'

class Dataset1(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [ ]:
dataloader = DataLoader(dataset, batch_size=16)

/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GAE, VGAE, GCNConv
import numpy as np

# Define the number of epochs to train for
num_epochs = 50

# Define the batch size for the dataloader
batch_size = 16

# Define the number of nodes and features in each node
num_nodes = 2160
num_features = 3

# Define the number of dimensions in the latent variable
hidden_dim = 16
output_dim = 6
input_dim = 3

# Define the device to use for training (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the encoder and decoder architectures for the VAE
class Encoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Encoder, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, output_dim)
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv_mu = GCNConv(hidden_dim, output_dim)
        self.conv_logstd = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # x = F.relu(self.lin1(x))
        # x = F.dropout(x, training=self.training)
        # x = self.lin2(x)
        # x = torch_geometric.utils.scatter('mean', x, edge_index)
        # return x
        x = self.conv1(x, edge_index).relu()
        # return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
        return self.conv_mu(x, edge_index)

class Decoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, output_dim)
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv_mu = GCNConv(hidden_dim, output_dim)
        self.conv_logstd = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # x = F.relu(self.lin1(x))
        # x = F.dropout(x, training=self.training)
        # x = self.lin2(x)
        # x = torch.matmul(edge_index, x)
        # return x
        x = self.conv1(x, edge_index).relu()
        # return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
        return self.conv_mu(x, edge_index)


class GraphAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, output_dim)
        self.decoder = Decoder(output_dim, hidden_dim, input_dim)

    def forward(self, x, edge_index):
        z = self.encoder(x, edge_index)
        x_hat = self.decoder(z, edge_index)
        return x_hat


# Define the VAE model
# class VAE(nn.Module):
#     def __init__(self):
#         super(VAE, self).__init__()
#         self.encoder = Encoder(input_dim, hidden_dim, output_dim)
#         self.decoder = Decoder(input_dim, hidden_dim, output_dim)
#         self.gae = VGAE(self.encoder, self.decoder)
        
#     def forward(self, x, edge_index):
#         z = self.encoder(x, edge_index)
#         x_hat = self.decoder(z, edge_index)
#         return x_hat

In [ ]:
import torch_geometric.utils as utils

dataloader = DataLoader(graph_list, batch_size=batch_size, shuffle=True)

# Initialize the VAE model and move it to the device
# model = VAE().to(device)
model = GraphAutoencoder(input_dim, hidden_dim, output_dim).to(device)

# Define the optimizer for the VAE
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Define the loss function for the VAE
def loss_function(output, x):
    # mean, logstd = output.z_mean, output.z_log_stddev
    # mean, logstd = output, 0
    # kl_loss = -0.5 * torch.sum(1 + logstd - mean.pow(2) - logstd.exp())
    x_recon = output #.x_recon
    recon_loss = F.mse_loss(x_recon, x)
    loss = recon_loss # + kl_loss
    return loss

# Train the VAE model
for epoch in range(num_epochs):
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        # edge_index = utils.batched_edge_indices(batch.edge_index, batch.batch).to(device)
        output = model(batch.x, batch.edge_index)
        loss = loss_function(output, batch.x)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss.item()}')

/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1/50: Loss=22752.716796875
Epoch 2/50: Loss=11014923264.0
Epoch 3/50: Loss=1707.9493408203125
Epoch 4/50: Loss=2536870248448.0
Epoch 5/50: Loss=1436.6904296875
Epoch 6/50: Loss=1303.1873779296875
Epoch 7/50: Loss=217741017481216.0
Epoch 8/50: Loss=1449.9478759765625
Epoch 9/50: Loss=1312.916748046875
Epoch 10/50: Loss=1745.0675048828125
Epoch 11/50: Loss=1216.617431640625
Epoch 12/50: Loss=1563.6595458984375
Epoch 13/50: Loss=1632.7408447265625
Epoch 14/50: Loss=1084.971435546875
Epoch 15/50: Loss=843358.5
Epoch 16/50: Loss=1175.5943603515625
Epoch 17/50: Loss=1212.5472412109375
Epoch 18/50: Loss=1333.5699462890625
Epoch 19/50: Loss=1222.5030517578125
Epoch 20/50: Loss=1028.2706298828125
Epoch 21/50: Loss=1422.88623046875
Epoch 22/50: Loss=150983803928576.0
Epoch 23/50: Loss=1173.6839599609375
Epoch 24/50: Loss=1370.26513671875
Epoch 25/50: Loss=1127.17333984375
Epoch 26/50: Loss=1006.363037109375
Epoch 27/50: Loss=1640.4342041015625
Epoch 28/50: Loss=27435085070336.0
Epoch 29/50

In [ ]:
state_dict = model.state_dict()

# Save the state dictionary to a file
torch.save(state_dict, 'my_model.pt')

In [ ]:
model = MyModel()

# Load the saved state dictionary into the model
state_dict = torch.load('my_model.pt')
model.load_state_dict(state_dict)

**Embed the graphs and save it:**

In [ ]:
# Embed the input graphs using the trained VAE
embeddings = []

for graph in graph_list:
    graph = graph.to(device)
    with torch.no_grad():
        output = model.encoder(graph.x, graph.edge_index)
    embeddings.append(output[0].cpu().numpy())

np.save('embeddings.npy', embeddings)

In [ ]:
embeddings = np.array(embeddings)

**Cluster the Graphs:**

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture

# Load the embedded graph data
# embeddings = np.load('embeddings.npy')

# Set the number of clusters
num_clusters = 10

# Initialize the GMM model
gmm = GaussianMixture(n_components=num_clusters)

# Fit the GMM model to the embedded data
gmm.fit(embeddings)

# Predict the cluster labels for the embedded data
cluster_labels = gmm.predict(embeddings)